In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import MultiPolygon, Point
import numpy as np
import pandas as pd
# Open the masked raster file
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
STATES = ["RS","SC","PR","SP","MG","ES","RJ"]
INFOS_UFS = gpd.read_file("assets/bcim_2016_21_11_2018.gpkg", layer = 'lim_unidade_federacao_a')[['sigla','geometry']]

INFOS_UFS = INFOS_UFS[INFOS_UFS['sigla'].isin(STATES)].reset_index()
br_union_geo = INFOS_UFS['geometry'].unary_union

In [ ]:
def filter_shape(data_path, geometry, filtered_data_path):
    with rasterio.open(data_path) as src:
        out_image, out_transform = mask(src, geometry, crop=True)
        out_image[out_image == src.nodata] = np.nan
    with rasterio.open(filtered_data_path, 'w', driver='GTiff', 
                   height=out_image.shape[1], width=out_image.shape[2], 
                   count=src.count, dtype=out_image.dtype, 
                   crs=src.crs, transform=out_transform) as dst:
        dst.write(out_image)
    return out_image

def turn_tiff_into_dataframe(data_path, resolution_reduction:int = 20, export_path:str|None = None):
    with rasterio.open(data_path) as src:
        # Read the masked raster data
        masked_data = src.read(1, masked=True)  # Masked=True to handle NaN values
        xmin, ymin, xmax, ymax = src.bounds
        
        n_rows, n_cols = masked_data.shape

    # Create grids of latitude and longitude values
    lons = np.linspace(xmin, xmax, n_cols)
    lats = np.linspace(ymax, ymin, n_rows)

    # Create a meshgrid of latitude and longitude values
    lon_grid, lat_grid = np.meshgrid(lons, lats)

    df = pd.DataFrame()

    lon_flat = lon_grid.flatten()[::resolution_reduction]
    lat_flat = lat_grid.flatten()[::resolution_reduction]
    masked_flat = masked_data.flatten()[::resolution_reduction]

    df['longitude'] = pd.Series(lon_flat)
    df['latitude'] = pd.Series(lat_flat)
    df['value'] = pd.Series(masked_flat)
    if export_path:
        df.to_parquet(export_path, index=False)
    return df

In [ ]:
# Open the GeoTIFF file
with rasterio.open('assets\wc2.1_30s_bio\wc2.1_30s_bio_1.tif') as src:
    # Read the MultiPolygon geometry
    geoms = list(INFOS_UFS['geometry'])
    nodata = src.nodata
    # Mask the raster with the MultiPolygon
    out_image, out_transform = mask(src, geoms, crop=True)
    out_image[out_image == nodata] = np.nan  # Set fill values to NaN

In [ ]:
# The resulting out_image is a numpy array containing the masked values from the GeoTIFF
# The out_transform contains the affine transformation information

# Optionally, you can save the masked data to a new GeoTIFF file
with rasterio.open('generated_files/masked_raster.tif', 'w', driver='GTiff', 
                   height=out_image.shape[1], width=out_image.shape[2], 
                   count=src.count, dtype=out_image.dtype, 
                   crs=src.crs, transform=out_transform) as dst:
    dst.write(out_image)


In [ ]:

with rasterio.open("generated_files\masked_raster.tif") as src:
    # Read the masked raster data
    masked_data = src.read(1, masked=True)  # Masked=True to handle NaN values
    xmin, ymin, xmax, ymax = src.bounds
    
n_rows, n_cols = masked_data.shape

# Create grids of latitude and longitude values
lons = np.linspace(xmin, xmax, n_cols)
lats = np.linspace(ymax, ymin, n_rows)

# Create a meshgrid of latitude and longitude values
lon_grid, lat_grid = np.meshgrid(lons, lats)

df = pd.DataFrame()

#lon_flat = lon_grid.flatten()
#lat_flat = lat_grid.flatten()
#masked_flat = masked_data.flatten()

lon_flat = lon_grid.flatten()[::20]
lat_flat = lat_grid.flatten()[::20]
masked_flat = masked_data.flatten()[::20]

df['longitude'] = pd.Series(lon_flat)
df['latitude'] = pd.Series(lat_flat)
df['value'] = pd.Series(masked_flat)

In [ ]:
df[['longitude','latitude','value']].to_parquet("generated_files/temperature_considering_latitude_longitude.parquet")

In [ ]:
df = pd.read_parquet("generated_files/temperature_considering_latitude_longitude.parquet")
df = df[~df['value'].isna()]
with rasterio.open("generated_files\masked_raster.tif") as src:
    # Get the spatial extent of the raster
    xmin, ymin, xmax, ymax = src.bounds

In [ ]:
df.shape

In [ ]:
radius=2,

In [ ]:
occurence = pd.read_parquet("assets/gibf.parquet")

In [ ]:
occurence

In [ ]:
occur_lat = occurence['decimalLatitude']
occur_lon = occurence['decimalLongitude']

In [ ]:
heatmap_fig = px.scatter_mapbox(
    df,
    lat="latitude",
    lon="longitude",
    color="value",
    center=dict(lat=(ymin + ymax) / 2, lon=(xmin + xmax) / 2),
    color_continuous_scale="Bluered",
    zoom=4,
    mapbox_style="carto-positron",  # Choose a mapbox style
    title='Ocorrência e temperatura',
    opacity=1
)

heatmap_fig.add_trace(go.Scattermapbox(
        lat=occur_lat,
        lon=occur_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=6,
            color='rgb(10, 10, 10)',
            opacity=0.4
        ),
        hoverinfo='text'
    ))

heatmap_fig.add_trace(go.Scattermapbox(
        lat=occur_lat,
        lon=occur_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            color='rgb(75, 173, 64)',
            opacity=0.4
        ),
        hoverinfo='none'
    ))

heatmap_fig.show()